# mô hình ALS (Alternating Least Squares)

In [27]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col, explode, split

from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
spark = SparkSession.builder.appName('AnimeRecommendation').getOrCreate()

data = spark.read.csv("./data/proceeded/anime_watched_with_details.csv", header=True, inferSchema=True)


In [29]:
data = data.select("user_id", "titles_watched", "genres")


In [30]:
genres_split = data.select("user_id", "titles_watched", "genres") \
    .withColumn("genres", split(col("genres"), ","))


In [31]:
genres_split = genres_split.withColumn("genres", explode(col("genres")))


In [32]:
indexer = StringIndexer(inputCol="genres", outputCol="genre_index")
indexed_genres = indexer.fit(genres_split).transform(genres_split)


In [33]:
encoder = OneHotEncoder(inputCol="genre_index", outputCol="genre_vec")
encoded_genres = encoder.fit(indexed_genres).transform(indexed_genres)


In [34]:
item_indexer = StringIndexer(inputCol="titles_watched", outputCol="item_id")
data_with_item_id = item_indexer.fit(encoded_genres).transform(encoded_genres)


In [35]:
data_with_item_id = data_with_item_id.dropna(subset=["genre_index", "genre_vec"])

data_ready = data_with_item_id.select("user_id", "item_id", "genre_index")

data_ready.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: double (nullable = false)
 |-- genre_index: double (nullable = false)



In [36]:
# Chia dữ liệu thành tập huấn luyện (80%) và kiểm tra (20%)
train_data, test_data = data_ready.randomSplit([0.8, 0.2], seed=42)

In [37]:
# Tạo mô hình ALS
als = ALS(userCol="user_id", itemCol="item_id", ratingCol="genre_index", coldStartStrategy="drop")

# Huấn luyện mô hình
model = als.fit(data_ready)

In [38]:
# Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)

In [39]:
# Loại bỏ các hàng không có giá trị prediction
predictions = predictions.dropna(subset=["prediction"])

In [40]:
# Đánh giá mô hình bằng RMSE
evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="genre_index", predictionCol="prediction")
rmse = evaluator_rmse.evaluate(predictions)

# Đánh giá mô hình bằng MAE
evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="genre_index", predictionCol="prediction")
mae = evaluator_mae.evaluate(predictions)

In [41]:
print(f"RMSE (Root Mean Squared Error): {rmse}")
print(f"MAE (Mean Absolute Error): {mae}")

RMSE (Root Mean Squared Error): 11.712391376183813
MAE (Mean Absolute Error): 8.757198941181052


In [42]:
# Dự đoán các anime cho một người dùng
user_id = 1  # Ví dụ cho user_id = 1
watched_items = data_ready.filter(col("user_id") == user_id).select("item_id").rdd.flatMap(lambda x: x).collect()

# Dự đoán top anime mà user có thể thích
all_recommendations = model.recommendForAllUsers(10)  # Gợi ý 10 phim cho tất cả người dùng
user_recommendations = all_recommendations.filter(col("user_id") == user_id)

# Rã các gợi ý ra từng dòng
user_recommendations = user_recommendations.select(col("user_id"), explode(col("recommendations")).alias("recommendation"))
user_recommendations = user_recommendations.select("user_id", col("recommendation.item_id").alias("item_id"), col("recommendation.rating").alias("prediction"))

# Lọc bỏ các phim user đã xem
user_recommendations = user_recommendations.filter(~col("item_id").isin(watched_items))

top_recommendations = user_recommendations.join(data_with_item_id, on="item_id", how="left")

top_recommendations = top_recommendations.select("titles_watched", "genres", "prediction")

top_recommendations = top_recommendations.orderBy(col("prediction").desc())

top_recommendations = top_recommendations.dropDuplicates(["titles_watched"])

top_5_recommendations = top_recommendations.limit(5)

top_5_recommendations.show(truncate=False)

+------------------------------------------------------+-----------+----------+
|titles_watched                                        |genres     |prediction|
+------------------------------------------------------+-----------+----------+
|Attack on Titan (Live-Action)                         |Live Action|101.36276 |
|Bima - Satria Garuda                                  |Tokusatsu  |83.40841  |
|Bách Hoa Liễu Loạn: Samurai Girls Specials            |Ecchi      |85.557556 |
|Cuộc Chiến Sinh Tử (Trò Chơi Sinh Tử)                 |Live Action|86.771255 |
|KAMEN RIDER X KAMEN RIDER W & DECADE – MOVIE WARS 2010|Tokusatsu  |86.50363  |
+------------------------------------------------------+-----------+----------+



In [49]:
import matplotlib.pyplot as plt
import pandas as pd
import os

model_path = "./models/anime_recommendation_model"

model.save(model_path)
print(f"Model saved at {model_path}")

Model saved at ./models/anime_recommendation_model
